In [19]:
def sort_priority(numbers, group):
    found = False
    def helper(x):
        if x in group:
            found = True
            return (0, x)
        else:
            return (1, x)
    numbers.sort(key=helper)
    return found

sort_priority(numbers, group) 함수는 group 에 있는 숫자들의 우선순위를 높혀서 정렬하는 숫자이다.

만약 numbers 에서 group 에 있는 숫자를 발견하면 found = True 로 변경하고 우선순위를 높이기 위한 튜플을 반환한다.

In [20]:
numbers = [4, 5, 2, 7, 3, 7]
group = [5, 7]
found = sort_priority(numbers, group)
print(numbers)
print(found)

[5, 7, 7, 2, 3, 4]
False


위의 결과를 보면 정렬은 잘되었지만 기대했던 found 값인 True가 반환되지 않고 False 를 반환했다.

이유는 클로저의 스코프 탐색 때문이다.

파이썬 인터프리터는 다음의 순서로 변수가 정의되어 있는 스코프를 탐색한다.

1. 현재 함수의 스코프
2. 현재 스코프를 감싸고 있는 스코프
3. 코드를 포함하고 있는 모듈으 스코프 (전역 스코프)
4. 내장 스코프 (len, str 같은 함수를 담고 있는 스코프)

파이썬은 변수가 현재 스코프에 존재하지 않으면 새 변수를 정의한다.
이 변수의 스코프는 이 변수를 할당한 함수 스코프이다.

```python
def sort_priority(numbers, group):
    found = False # sort_priority 스코프
    def helper(x):
        found = True
        if x in group:
            found = True  # helper 의 스코프
            return (0, x)
        else:
            return (1, x)
    numbers.sort(key=helper)
    return found
```

helper 함수 안에

```python
found = True
```

라인은 False 인 found 변수의 값을 True 로 변경하는 것이 아닌, 새로운 found 변수를 생성하는 것이다.

nonlocal 키워드를 사용하면 클로저에서 데이터를 얻어올수 있다.
nonlocal '변수이름' 문법을 사용하면 '변수이름' 에 할당할 때 스코프 탐색이 일어나야 함을 나타낸다.

In [21]:
def sort_priority(numbers, group):
    found = False
    def helper(x):
        nonlocal found
        if x in group:
            found = True
            return (0, x)
        else:
            return (1, x)
    numbers.sort(key=helper)
    return found

In [22]:
numbers = [4, 5, 2, 7, 3, 7]
group = [5, 7]
found = sort_priority(numbers, group)
print(numbers)
print(found)

[5, 7, 7, 2, 3, 4]
True


하지만 전역 변수를 안티패턴과 마찬가지로 nonlocal 키워드는 간단한 함수 이외에 사용하지 않도록 주의해야 한다.

위의 문제는 __call__ 라는 특별한 메서드를 사용하면 해결할 수 있다.

In [24]:
class Sorter(object):
    def __init__(self, group):
        self.group = group
        self.found = False
        
    def __call__(self, x):
        if x in self.group:
            self.found = True
            return (0, x)
        else:
            return (1, x)
        
numbers = [4, 5, 2, 7, 3, 7]
group = [5, 7]

sorter = Sorter(group)
numbers.sort(key=sorter)
found = sorter.found
print(numbers)
print(found)

[5, 7, 7, 2, 3, 4]
True


파이썬2 에서는 nonlocal 키워드를 지원하지 않는다.

In [25]:
def sort_priority(numbers, group):
    found = [False]
    def helper(x):
        if x in group:
            found[0] = True
            return (0, x)
        else:
            return (1, x)
    numbers.sort(key=helper)
    return found[0]

numbers = [4, 5, 2, 7, 3, 7]
group = [5, 7]
found = sort_priority(numbers, group)
print(numbers)
print(found)

[5, 7, 7, 2, 3, 4]
True


대신에 found 를 mutable 한 list로 바꿔서 해결할 수 있다.